In [ ]:
import importlib
import sys
import torch
import pickle
import os

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')


from model.dropout_uncertainty_enc_dec_LSTM.dropout_uncertainty_model import DropoutUncertaintyEncoderDecoderLSTM

In [ ]:
#load model
file_path_model = '../../../training_variational_dropout/PCR-Test/PCR_full_grad_norm_2layer.pkl'
model = DropoutUncertaintyEncoderDecoderLSTM.load(file_path_model, dropout=0.1)

# Load the dataset
file_path_data_set = '../../../../../encoded_data/PCR_5_test.pkl'
pcr_test_dataset = torch.load(file_path_data_set, weights_only=False)

Data set categories:  ([('concept:name', 10, {'Callback timeout': 1, 'EOS': 2, 'Export result': 3, 'Export to EMS': 4, 'Match patient data': 5, 'Receive sample state': 6, 'Send notification': 7, 'Wait for plate validation': 8, 'timeout': 9}), ('cpee:lifecycle:transition', 6, {'EOS': 1, 'activity/calling': 2, 'activity/done': 3, 'activity/receiving': 4, 'dataelements/change': 5})], [('case_elapsed_time', 1, {}), ('event_elapsed_time', 1, {}), ('day_in_week', 1, {}), ('seconds_in_day', 1, {})])
Encoder input features:  [['concept:name', 'cpee:lifecycle:transition'], ['case_elapsed_time', 'event_elapsed_time', 'day_in_week', 'seconds_in_day']]
Decoder input+output features:  [['concept:name'], ['case_elapsed_time']]


Sequence length of decoder output:  4


Cells hidden size:  128
Number of LSTM layer:  4
Dropout rate:  0.1


Encoder number of labels for each input feature (categorical, numerical):  [[10, 6], [1, 1, 1, 1]]
Encoder indices of tensors in dataset used as input:  [[0, 1], [0,

In [3]:
import evaluation.probabilistic_evaluation
importlib.reload(evaluation.probabilistic_evaluation)
from evaluation.probabilistic_evaluation import ProbabilisticEvaluation

new_eval = ProbabilisticEvaluation(model, 
                                   pcr_test_dataset,
                                   concept_name='concept:name',
                                   num_processes=16,
                                   #growing_num_values = [],
                                   growing_num_values = ['case_elapsed_time'],
                                   samples_per_case = 1000,
                                   sample_argmax = False,
                                   use_variance_cat = True,
                                   use_variance_num = True)


In [ ]:
def save_chunk(results, i):
    chunk_number = (i + 1)
    filename = os.path.join(output_dir, f'results_part_{chunk_number:03d}.pkl')
    with open(filename, 'wb') as f:
        pickle.dump(results, f)
    print(f"Saved {len(results)} results to {filename}")
      
output_dir = '../../../../../../../evaluation_results/PCR/gn/'  

save_every = 50
results = {}
for i, (case_name, prefix_len, prefix, predicted_suffixes, suffix, mean_prediction) in enumerate(new_eval.evaluate_multi_processing(random_order=False)):
    print(case_name, prefix_len)
    assert((case_name, prefix_len) not in results)
    results[(case_name, prefix_len)] = (prefix, suffix, mean_prediction, predicted_suffixes)
    print(prefix_len, len(suffix))
    if (i + 1) % save_every == 0:
        save_chunk(results, i)
        results = {}

if len(results):
    save_chunk(results, i)

  0%|          | 0/998 [00:00<?, ?it/s]

359 16
16 2
359 15
15 3
359 14
14 4
359 13
13 5
359 12
12 6
359 11
11 7
359 10
10 8
359 9
9 9
359 8
8 10
359 7
7 11
359 6
6 12
359 2
2 16
359 5
5 13
359 3
3 15
359 1
1 17
359 4
4 14
359 17
17 1
364 15
15 3
364 14
14 4
364 13
13 5
364 1
1 17
364 12
12 6
364 2
2 16
364 11
11 7
364 10
10 8
364 3
3 15
364 6
6 12
364 5
5 13
364 4
4 14
364 9
9 9
364 8
8 10
364 7
7 11
364 16
16 2
364 17
17 1
366 14
14 4
366 13
13 5
366 12
12 6
366 11
11 7
366 10
10 8
366 1
1 17
366 15
15 3
366 9
9 9
366 2
2 16
366 6
6 12
366 3
3 15
366 5
5 13
366 8
8 10
366 7
7 11
366 4
4 14
366 16
16 2
Saved 50 results to ../../../../../evaluation_results/PCR/gn/pos/results_part_050.pkl
366 17
17 1
372 14
14 4
372 13
13 5
372 12
12 6
372 11
11 7
372 1
1 17
372 10
10 8
372 2
2 16
372 9
9 9
372 3
3 15
372 7
7 11
372 8
8 10
372 5
5 13
372 6
6 12
372 4
4 14
372 15
15 3
372 16
16 2
372 17
17 1
373 14
14 4
373 13
13 5
373 12
12 6
373 1
1 17
373 11
11 7
373 10
10 8
373 9
9 9
373 2
2 16
373 7
7 11
373 6
6 12
373 5
5 13
373 3
3 15
37